# Build a SeqData

In [1]:
# Imports
import os
import numpy as np
import seqdata as sd

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
# TODO: change to your path
data_dir = "/cellar/users/aklie/data/datasets/K562_CTCF-ChIP-seq/data"

In [3]:
# Set-up all file paths
peaks = os.path.join(data_dir, "K562_CTCF-ChIP-seq_peaks.bed")
signals = [
    os.path.join(data_dir, "ENCSR000AKO_plus_counts.bw"),
    os.path.join(data_dir, "ENCSR000AKO_minus_counts.bw")
]
sample_names = ['signal+', 'signal-']
fasta = os.path.join(data_dir, "hg38.fa")
peaks, signals

('/cellar/users/aklie/data/datasets/K562_CTCF-ChIP-seq/data/peaks.bed',
 ['/cellar/users/aklie/data/datasets/K562_CTCF-ChIP-seq/data/plus.bw',
  '/cellar/users/aklie/data/datasets/K562_CTCF-ChIP-seq/data/minus.bw'])

In [4]:
# Make output directory if doesn't exist
out = os.path.join(data_dir, "K562_CTCF-ChIP-seq.zarr")
out

'/cellar/users/aklie/data/datasets/K562_CTCF-ChIP-seq/data/K562_CTCF-ChIP-seq.zarr'

In [5]:
# Compose a SeqData from a set of files - only run this once!
sdata = sd.from_region_files(
    sd.GenomeFASTA(
        'seq',
        fasta,
        batch_size=2048,
        n_threads=2,
    ),
    sd.BigWig(
        'cov',
        signals,
        sample_names,
        batch_size=2048,
        n_jobs=2,
        threads_per_job=2,
    ),
    path=out,
    fixed_length=2114,
    bed=peaks,
    overwrite=True,
    max_jitter=128
)

100%|██████████| 51759/51759 [02:15<00:00, 381.68it/s] 
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
100%|██████████| 51759/51759 [00:10<00:00, 5149.47it/s]


# Check against pyBigWig

In [6]:
import pyBigWig
import matplotlib.pyplot as plt
import seaborn as sns

def plot_tracks(tracks, interval, height=1.5, colors=None):
  _, axes = plt.subplots(len(tracks), 1, figsize=(20, height * len(tracks)), sharex=True)
  if not isinstance(axes, np.ndarray):
    axes = [axes]
  for ax, (title, y) in zip(axes, tracks.items()):
    if colors is not None:
      ax.fill_between(np.linspace(interval["start"], interval["end"], num=len(y)), y, color=colors[title])
    else:
      ax.fill_between(np.linspace(interval["start"], interval["end"], num=len(y)), y)
    ax.set_title(title)
    sns.despine(top=True, right=True, bottom=True)
  ax.set_xlabel(f"{interval['chrom']}:{interval['start']}-{interval['end']}")
  plt.tight_layout()

In [7]:
# The actual BigWig file
plus_file = pyBigWig.open(str(signals[0]))
minus_file = pyBigWig.open(str(signals[1]))

In [19]:
# Get data
seq_num = 10
ser = sdata[["name", "chrom", "chromStart", "chromEnd"]].to_dataframe().loc[seq_num]
sdata_vals = sdata["cov"][seq_num].values
pybw_plus_vals = np.nan_to_num(np.array(plus_file.values(ser["chrom"], ser["chromStart"], ser["chromEnd"])))
pybw_minus_vals = np.nan_to_num(np.array(minus_file.values(ser["chrom"], ser["chromStart"], ser["chromEnd"])))
chrom = ser["chrom"]
chromStart = ser["chromStart"]
chromEnd = ser["chromEnd"]
interval = dict(chrom=chrom, start=chromStart, end=chromEnd)
tracks = {
    "SeqData K562 CTCF ChIP-seq (+)": np.squeeze(sdata_vals[0]),
    "SeqData K562 CTCF ChIP-seq (-)": np.squeeze(sdata_vals[1]),
    "pyBigWig K562 CTCF ChIP-seq (+)": np.squeeze(pybw_plus_vals),
    "pyBigWig K562 CTCF ChIP-seq (-)": np.squeeze(pybw_minus_vals),

} 
colors={
    "SeqData K562 CTCF ChIP-seq (+)": "lightblue",
    "SeqData K562 CTCF ChIP-seq (-)": "lightcoral",
    "pyBigWig K562 CTCF ChIP-seq (+)": "lightblue",
    "pyBigWig K562 CTCF ChIP-seq (-)": "lightcoral",
}

# Plot tracks
plot_tracks(
    tracks=tracks,
    interval=interval,
    colors=colors
)

# DONE!

---